In [47]:
import pandas as pd
import json
import numpy as np    

# Load basic CI relational data
ci_df = pd.read_json("../sql/Configuration_Item.json")
attributes_df = pd.read_json("../sql/Attribute.json")

# Load performance relational data
rules_df = pd.read_json("../sql/perfRules.json")
rule_instances_df = pd.read_json("../sql/perfRuleInstances.json")
perf_data_df = pd.read_json("../sql/perfRuleData.json")

# Load monitors relational data
monitors_df = pd.read_json("../sql/Monitors.json")
monitor_instances_df = pd.read_json("../sql/MonitorInstances.json")
monitor_instance_states_df=pd.read_json("../sql/MonitorInstanceStates.json")
health_states_df=pd.read_json("../sql/HealthStates.json").set_index('state_id')

# Load alert instances relational data
alert_instances_df = pd.read_json("../sql/AlertInstances.json")
alert_states_df = pd.read_json("../sql/AlertStates.json").set_index("state_id")



# Merge CI and attributes data into one dataframe
ci_att_df = pd.merge(ci_df,attributes_df,on="ci_id")
cp=ci_att_df.pivot(index="name", columns="att_name")
cp.columns = ['ci_id','ci_id2','ci_id3','device_type','env','ip_address']
cp.drop(columns=['ci_id2', 'ci_id3'], inplace=True)
configuration_items=cp.reset_index()
monitor_instances_df = pd.merge(monitor_instances_df,monitors_df,on="monitor_id")


# Merge performance dataframes
rule_instances_df.rename(columns={'per_ruleId':'ruleId'}, inplace=True)
rule_instances_df = pd.merge(ci_df,rule_instances_df,on="ci_id")
rule_instances = pd.merge(rule_instances_df,rules_df,on="ruleId")
rule_instances.sort_values('ci_id')


alert_instances_df = pd.merge(alert_instances_df,monitor_instances_df,on="mon_isntance_id" )
alert_instances_df = pd.merge(alert_instances_df,ci_df, on="ci_id")
alert_instances_df = alert_instances_df.loc[:,['alert_name','alert_description','name_y','state_change_date','priority']]
alert_instances_df.rename(columns={'name_y':'configuration_item'}, inplace=True)



In [49]:
configuration_item_documents= []
alert_documents = []

for ci_index, configuration_item in configuration_items.iterrows():
    ci = json.loads(configuration_item.to_json())
    
    # Attach performance rules instances
    ci["performance_rules"]=[]
    for rule_index, rule_instance in rule_instances[rule_instances["ci_id"]==ci["ci_id"]].loc[:,'ruleId':'rule_description'].iterrows():
        rule = json.loads(rule_instance.to_json())
        
        # Attach performance data 
        data = []
        for perf_data_index, perf_data in perf_data_df[perf_data_df["perf_rule_instId"]==rule["ruleId"]].loc[:,'value':'date'].iterrows():
            d = json.loads(perf_data.to_json())
            data.append(d)
        rule["data"]=data
        ci["performance_rules"].append(rule)
        
    # Attach monitor instances
    ci["monitors"]= []
    
    for monitor_index, monitor_instance in monitor_instances_df[monitor_instances_df["ci_id"]==ci["ci_id"]].loc[:,['mon_isntance_id','name','description']].iterrows():
        monitor = json.loads(monitor_instance.to_json())
        
        # Attach health state for each monitor instance
        states = []
        
        for monitor_instance_index, monitor_instance_state in monitor_instance_states_df[monitor_instance_states_df["mon_isntance_id"]==monitor["mon_isntance_id"]].loc[:,['state_change_date','health_State_id']].iterrows():
            state_id=int(monitor_instance_state["health_State_id"])-1
            monitor_instance_state["health_State_id"] = health_states_df.iloc[state_id].state_name
            currentState = json.loads(monitor_instance_state.to_json())
            states.append(currentState)
        
        monitor["states"]=states
        
        ci["monitors"].append(monitor)
    
    configuration_item_documents.append(ci)
   
for alert_index, alert_row in alert_instances_df.iterrows():
    alert = json.loads(alert_row.to_json())
    priority_id=int(alert_row["priority"])-1
    alert['priority']= alert_states_df.iloc[priority_id].state_name
    alert_documents.append(alert)
    
    



In [50]:
with open('../nosql/cis.json', 'w') as fout:
    fout.write(json.dumps(configuration_item_documents, indent=4))
with open('../nosql/alerts.json', 'w') as fout:
    fout.write(json.dumps(alert_documents, indent=4))

    

In [29]:
alert_instances_df

,alert_name,alert_description,configuration_item,state_change_date,severity
0,Heartbeat failure,The device is not sending heartbeats,impossiblejamb.local,2020-05-23T03:44:08,1
1,Heartbeat failure,The device is not sending heartbeats,rubberyclock.local,2020-05-23T03:44:37,1
2,Heartbeat failure,The device is not sending heartbeats,wrytug.local,2020-05-23T03:44:27,1
3,Heartbeat failure,The device is not sending heartbeats,worstblip.local,2020-05-23T03:44:12,1
4,Heartbeat failure,The device is not sending heartbeats,lastbuyer.local,2020-05-23T03:44:21,1
...,...,...,...,...,...
95,Heartbeat failure,The device is not sending heartbeats,cutestep.local,2020-05-23T03:44:37,1
96,Heartbeat failure,The device is not sending heartbeats,worrisomeflare.local,2020-05-23T03:44:16,1
97,Heartbeat failure,The device is not sending heartbeats,vivaciousjunk.local,2020-05-23T03:44:25,1
98,Heartbeat failure,The device is not sending heartbeats,celebratedday.local,2020-05-23T03:44:51,1


In [14]:
monitor_instances_df

,mon_isntance_id,ci_id,monitor_id,name,description,alert_name,alert_description
0,1,1,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats
1,2,2,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats
2,3,3,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats
3,4,4,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats
4,5,5,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats
...,...,...,...,...,...,...,...
785,670,90,10,Operating System Total Percent Interrupt Time,The total percentage of interrupt time (for al...,Total Percentage Interrupt Time is too high,The threshold for the Processor\\% Interrupt T...
786,700,100,10,Operating System Total Percent Interrupt Time,The total percentage of interrupt time (for al...,Total Percentage Interrupt Time is too high,The threshold for the Processor\\% Interrupt T...
787,730,37,10,Operating System Total Percent Interrupt Time,The total percentage of interrupt time (for al...,Total Percentage Interrupt Time is too high,The threshold for the Processor\\% Interrupt T...
788,760,72,10,Operating System Total Percent Interrupt Time,The total percentage of interrupt time (for al...,Total Percentage Interrupt Time is too high,The threshold for the Processor\\% Interrupt T...


In [26]:
alert_instances_df3

,alert_instance_id,severity,priority,alert_State_id,state_change_date,mon_isntance_id,ci_id,monitor_id,name_x,description,alert_name,alert_description,configuration_item
0,1,1,1,1,2020-05-23T03:44:08,1,1,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,impossiblejamb.local
1,2,1,1,1,2020-05-23T03:44:37,2,2,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,rubberyclock.local
2,3,1,1,1,2020-05-23T03:44:27,3,3,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,wrytug.local
3,4,1,1,1,2020-05-23T03:44:12,4,4,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,worstblip.local
4,5,1,1,1,2020-05-23T03:44:21,5,5,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,lastbuyer.local
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1,1,1,2020-05-23T03:44:37,96,96,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,cutestep.local
96,97,1,1,1,2020-05-23T03:44:16,97,97,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,worrisomeflare.local
97,98,1,1,1,2020-05-23T03:44:25,98,98,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,vivaciousjunk.local
98,99,1,1,1,2020-05-23T03:44:51,99,99,31,Heartbeat Monitor,This monitor checks if the device is up,Heartbeat failure,The device is not sending heartbeats,celebratedday.local


In [39]:
alert_states_df


,state_name
state_id,
1,New
2,Assigned
3,Closed
